In [56]:
import pandas as pd

In [57]:
from dataclasses import dataclass


@dataclass(frozen=True)
class Observation:
    hr: int
    bp: int
    temp: float
    resp: float
    ox_sats: float
    insp_ox: float

In [58]:
obs_1 = Observation(hr=52, bp=102, temp=36.2, resp=22, ox_sats=94, insp_ox=30)

<h1> Membership Functions

<h1> Fuzzy Variables

In [59]:
class custom_mf_7_var:
    def __init__(self, path):
        self.path = path
        self.df = pd.read_csv(path)
        
        keys = self.df.loc[:, 'Value'].values
        
        self.B_SevC = dict(zip(keys, self.df.loc[:, 'Below normal - severe concern'].values))
        self.B_ModC = dict(zip(keys, self.df.loc[:, 'Below normal - moderate concern'].values))
        self.B_MildC = dict(zip(keys, self.df.loc[:, 'Below normal - mild concern'].values))
        self.no_con = dict(zip(keys, self.df.loc[:,'No concern'].values))
        self.A_MildC = dict(zip(keys, self.df.loc[:, 'Above normal - mild concern'].values))
        self.A_ModC = dict(zip(keys, self.df.loc[:, 'Above normal - moderate concern'].values))
        self.A_SevC = dict(zip(keys, self.df.loc[:, 'Above normal - severe concern'].values))
        
        self.fs = [self.B_SevC, self.B_ModC, self.B_MildC, self.no_con, self.A_MildC, self.A_ModC, self.A_SevC]
        self.labels = ['Below normal - severe concern', 'Below normal - moderate concern', 
                       'Below normal - mild concern', 'No concern', 
                       'Above normal - mild concern', 'Above normal - moderate concern', 
                       'Above normal - severe concern']
    
    def __call__(self, inp):
        out = {}
        for label, fs in zip(self.labels, self.fs):
            membership_value = fs.get(inp, 0.0)
            out[label] = membership_value
        
        return out
    
class custom_mf_3_var_up:
    def __init__(self, path):
        self.path = path
        self.df = pd.read_csv(path)
        
        keys = self.df.loc[:, 'Value'].values
        
        self.no_con = dict(zip(keys, self.df.loc[:,'No concern'].values))
        self.A_MildC = dict(zip(keys, self.df.loc[:, 'Above normal - mild concern'].values))
        self.A_ModC = dict(zip(keys, self.df.loc[:, 'Above normal - moderate concern'].values))
        self.A_SevC = dict(zip(keys, self.df.loc[:, 'Above normal - severe concern'].values))
        
        self.fs = [self.no_con, self.A_MildC, self.A_ModC, self.A_SevC]
        self.labels = ['No concern', 
                       'Above normal - mild concern', 'Above normal - moderate concern', 
                       'Above normal - severe concern']
    
    def __call__(self, inp):
        out = {}
        for label, fs in zip(self.labels, self.fs):
            membership_value = fs.get(inp, 0.0)
            out[label] = membership_value
        
        return out

        
class custom_mf_3_var_down:
    def __init__(self, path):
        self.path = path
        self.df = pd.read_csv(path)
        
        keys = self.df.loc[:, 'Value'].values
        
        self.B_SevC = dict(zip(keys, self.df.loc[:, 'Below normal - severe concern'].values))
        self.B_ModC = dict(zip(keys, self.df.loc[:, 'Below normal - moderate concern'].values))
        self.B_MildC = dict(zip(keys, self.df.loc[:, 'Below normal - mild concern'].values))
        self.no_con = dict(zip(keys, self.df.loc[:,'No concern'].values))

        self.fs = [self.B_SevC, self.B_ModC, self.B_MildC, self.no_con]
        self.labels = ['Below normal - severe concern', 'Below normal - moderate concern', 
                       'Below normal - mild concern', 'No concern']
    
    def __call__(self, inp):
        out = {}
        for label, fs in zip(self.labels, self.fs):
            membership_value = fs.get(inp, 0.0)
            out[label] = membership_value
        
        return out

        
class output_mf:
    def __init__(self, path):
        self.path = path
        self.df = pd.read_csv(path)
        
        keys = self.df.loc[:, 'Value'].values
        
        self.Sev = dict(zip(keys, self.df.loc[:, 'Severe concern'].values))
        self.Mod = dict(zip(keys, self.df.loc[:, 'Moderate concern'].values))
        self.Mild = dict(zip(keys, self.df.loc[:, 'Mild concern'].values))
        self.no_con = dict(zip(keys, self.df.loc[:,'No concern'].values))

        self.fs = [self.Sev, self.Mod, self.Mild, self.no_con]
        self.labels = ['Severe concern', 'Moderate concern', 
                       'Mild concern', 'No concern']
    
    def __call__(self, inp):
        out = {}
        for label, fs in zip(self.labels, self.fs):
            membership_value = fs.get(inp, 0.0)
            out[label] = membership_value
        
        return out

In [60]:

heart_rate = custom_mf_7_var('data/membership_function_plots/csv_data/heart_rate_membership_functions.csv')
blood_pressure = custom_mf_7_var('data/membership_function_plots/csv_data/systolic_blood_pressure_membership_functions.csv')
temperature = custom_mf_7_var('data/membership_function_plots/csv_data/temperature_membership_functions.csv')
respiratory_rate = custom_mf_7_var('data/membership_function_plots/csv_data/respiratory_rate_membership_functions.csv')
oxygen_saturation = custom_mf_3_var_down('data/membership_function_plots/csv_data/oxygen_saturation_membership_functions.csv')
inspired_oxygen = custom_mf_3_var_up('/home/msxtk13/Documents/FuzzySystemMain/data/membership_function_plots/csv_data/inspired_oxygen_concentration_membership_functions.csv')



def firings(hr, bp, temp, resp, ox, insp):
    firings = {
        'heart rate' : heart_rate(hr),
        'blood_pressure' : blood_pressure(bp),
        'temp' : temperature(temp),
        'resp' : respiratory_rate(resp),
        'oxygen sats' : oxygen_saturation(ox),
        'inspired_oxygen' : inspired_oxygen(insp)
    }
    return firings
    
res = firings(obs_1.hr, obs_1.bp, obs_1.temp, obs_1.resp, obs_1.ox_sats, obs_1.insp_ox)
res


{'heart rate': {'Below normal - severe concern': np.float64(0.0),
  'Below normal - moderate concern': np.float64(0.6),
  'Below normal - mild concern': np.float64(0.4),
  'No concern': np.float64(0.0),
  'Above normal - mild concern': np.float64(0.0),
  'Above normal - moderate concern': np.float64(0.0),
  'Above normal - severe concern': np.float64(0.0)},
 'blood_pressure': {'Below normal - severe concern': np.float64(0.0),
  'Below normal - moderate concern': np.float64(0.0),
  'Below normal - mild concern': np.float64(1.0),
  'No concern': np.float64(0.0),
  'Above normal - mild concern': np.float64(0.0),
  'Above normal - moderate concern': np.float64(0.0),
  'Above normal - severe concern': np.float64(0.0)},
 'temp': {'Below normal - severe concern': np.float64(0.0),
  'Below normal - moderate concern': np.float64(0.3333333333331281),
  'Below normal - mild concern': np.float64(0.6666666666668719),
  'No concern': np.float64(0.0),
  'Above normal - mild concern': np.float64(0.0),

<h1> Inference Engine

In [61]:
def inf_eng(res):
    
    no_concern = []
    mild_concern = []
    moderate_concern = []
    severe_concern = []
    
    for vital_name, vital_memberships in res.items():
        for key, value in vital_memberships.items():
            key_lower = key.lower()
            if 'severe' in key_lower:
                severe_concern.append(value)
            elif 'moderate' in key_lower:
                moderate_concern.append(value)
            elif 'mild' in key_lower:
                mild_concern.append(value)
            elif 'no concern' in key_lower:
                no_concern.append(value)
           
    
    levels_of_concern = {}
    levels_of_concern['No concern'] = max(no_concern) if no_concern else 0
    levels_of_concern['Mild concern'] = max(mild_concern) if mild_concern else 0
    levels_of_concern['Moderate concern'] = max(moderate_concern) if moderate_concern else 0
    levels_of_concern['Severe concern'] = max(severe_concern) if severe_concern else 0
    
    return levels_of_concern

inf_res = inf_eng(res)
inf_res

{'No concern': np.float64(0.0),
 'Mild concern': np.float64(1.0),
 'Moderate concern': np.float64(0.6),
 'Severe concern': np.float64(0.2131979695431472)}

<h1> DeFuzz

In [85]:
output = output_mf('data/membership_function_plots/csv_data/concern_output_membership_functions_ORIGINAL.csv')
output(50)

def de_fuzz(x):
    numerator = 0
    denominator = 0
    
    # Iterate through all possible output values (0-100)
    for i in range(0, 101):
        # Get membership values for this output point
        output_memberships = output(i)
        
        # Aggregate: take minimum of inference result and membership function, then max across all categories
        aggregated_membership = 0
        for concern_level, inference_value in x.items():
            # Get the membership value for this concern level at point i
            membership_at_i = output_memberships.get(concern_level, 0)
            # Clip the membership function at the inference firing strength
            clipped_value = min(inference_value, membership_at_i)
            # Take the maximum across all concern levels (aggregation)
            aggregated_membership = max(aggregated_membership, clipped_value)
        
        # Calculate centroid components
        numerator += i * aggregated_membership
        denominator += aggregated_membership
    
    # Avoid division by zero
    if denominator == 0:
        return 0
    
    centroid = numerator / denominator
    return centroid

de_fuzz(inf_res)


np.float64(41.43354453426559)

In [103]:
# NEWS2 Score Calculator
def calculate_news2(hr, bp, temp, resp, ox_sats, insp_ox):
    score = 0
    
    # Respiratory Rate
    if resp <= 8:
        score += 3
    elif resp <= 11:
        score += 1
    elif resp <= 20:
        score += 0
    elif resp <= 24:
        score += 2
    else:
        score += 3
    
    # Oxygen Saturation
    if ox_sats <= 91:
        score += 3
    elif ox_sats <= 93:
        score += 2
    elif ox_sats <= 95:
        score += 1
    else:
        score += 0
    
    # Supplemental Oxygen (inspired oxygen > 21 means on oxygen)
    if insp_ox > 21:
        score += 2
    
    # Temperature
    if temp <= 35.0:
        score += 3
    elif temp <= 36.0:
        score += 1
    elif temp <= 38.0:
        score += 0
    elif temp <= 39.0:
        score += 1
    else:
        score += 2
    
    # Systolic Blood Pressure
    if bp <= 90:
        score += 3
    elif bp <= 100:
        score += 2
    elif bp <= 110:
        score += 1
    elif bp <= 219:
        score += 0
    else:
        score += 3
    
    # Heart Rate
    if hr <= 40:
        score += 3
    elif hr <= 50:
        score += 1
    elif hr <= 90:
        score += 0
    elif hr <= 110:
        score += 1
    elif hr <= 130:
        score += 2
    else:
        score += 3
    
    return score

# Test cases covering different severity levels
test_cases = [
    # Low risk / Normal patients
    Observation(hr=75, bp=120, temp=36.8, resp=16, ox_sats=98, insp_ox=21),
    Observation(hr=80, bp=118, temp=36.5, resp=18, ox_sats=97, insp_ox=21),
    Observation(hr=70, bp=125, temp=37.0, resp=14, ox_sats=99, insp_ox=21),
    
    # Mild concern
    Observation(hr=52, bp=102, temp=36.2, resp=22, ox_sats=94, insp_ox=30),
    Observation(hr=95, bp=108, temp=37.5, resp=21, ox_sats=95, insp_ox=21),
    Observation(hr=48, bp=115, temp=36.0, resp=19, ox_sats=96, insp_ox=21),
    
    # Moderate concern
    Observation(hr=105, bp=95, temp=38.2, resp=23, ox_sats=93, insp_ox=28),
    Observation(hr=115, bp=98, temp=37.8, resp=24, ox_sats=92, insp_ox=35),
    Observation(hr=42, bp=92, temp=35.8, resp=22, ox_sats=94, insp_ox=40),
    Observation(hr=100, bp=88, temp=38.5, resp=25, ox_sats=91, insp_ox=40),
    
    # High concern
    Observation(hr=125, bp=85, temp=38.8, resp=26, ox_sats=90, insp_ox=50),
    Observation(hr=135, bp=82, temp=39.2, resp=28, ox_sats=89, insp_ox=60),
    Observation(hr=38, bp=78, temp=35.2, resp=27, ox_sats=88, insp_ox=50),
    
    # Severe concern
    Observation(hr=145, bp=75, temp=39.5, resp=30, ox_sats=86, insp_ox=70),
    Observation(hr=150, bp=70, temp=40.0, resp=32, ox_sats=85, insp_ox=80),
    Observation(hr=35, bp=68, temp=34.8, resp=8, ox_sats=84, insp_ox=85),
    
    # Mixed severity patterns
    Observation(hr=110, bp=105, temp=37.2, resp=20, ox_sats=95, insp_ox=24),
    Observation(hr=65, bp=130, temp=38.0, resp=23, ox_sats=92, insp_ox=28),
    Observation(hr=120, bp=160, temp=36.5, resp=25, ox_sats=93, insp_ox=30),
    Observation(hr=55, bp=95, temp=38.3, resp=19, ox_sats=96, insp_ox=35),
]

# Process all test cases
comparison_results = []
for i, obs in enumerate(test_cases):
    # Fuzzy system
    firing_res = firings(obs.hr, obs.bp, obs.temp, obs.resp, obs.ox_sats, obs.insp_ox)
    inference_res = inf_eng(firing_res)
    fuzzy_output = de_fuzz(inference_res)
    
    # NEWS2
    news2_score = calculate_news2(obs.hr, obs.bp, obs.temp, obs.resp, obs.ox_sats, obs.insp_ox)
    
    comparison_results.append({
        'Test': i + 1,
        'HR': obs.hr,
        'BP': obs.bp,
        'Temp': obs.temp,
        'Resp': obs.resp,
        'O2': obs.ox_sats,
        'InspO2': obs.insp_ox,
        'NEWS2': news2_score,
        'Fuzzy': round(fuzzy_output, 2)
    })

# Create comparison DataFrame
df_comparison = pd.DataFrame(comparison_results)

print("Fuzzy System vs NEWS2 Score Comparison")
print("=" * 80)
print(df_comparison.to_string(index=False))
print("\n" + "=" * 80)
print(f"Fuzzy Output Range: {df_comparison['Fuzzy'].min():.2f} - {df_comparison['Fuzzy'].max():.2f}")
print(f"NEWS2 Score Range: {df_comparison['NEWS2'].min()} - {df_comparison['NEWS2'].max()}")
print(f"\nCorrelation: {df_comparison['NEWS2'].corr(df_comparison['Fuzzy']):.3f}")

Fuzzy System vs NEWS2 Score Comparison
 Test  HR  BP  Temp  Resp  O2  InspO2  NEWS2  Fuzzy
    1  75 120  36.8    16  98      21      0  16.16
    2  80 118  36.5    18  97      21      0  16.16
    3  70 125  37.0    14  99      21      0  16.16
    4  52 102  36.2    22  94      30      6  41.43
    5  95 108  37.5    21  95      21      5  16.16
    6  48 115  36.0    19  96      21      2  26.19
    7 105  95  38.2    23  93      28     10  36.90
    8 115  98  37.8    24  92      35     10  44.01
    9  42  92  35.8    22  94      40      9  51.23
   10 100  88  38.5    25  91      40     13  50.03
   11 125  85  38.8    26  90      50     14  66.67
   12 135  82  39.2    28  89      60     16  66.67
   13  38  78  35.2    27  88      50     15  66.67
   14 145  75  39.5    30  86      70     16  76.40
   15 150  70  40.0    32  85      80     16  76.40
   16  35  68  34.8     8  84      85     17  70.10
   17 110 105  37.2    20  95      24      5  27.34
   18  65 130  38.0    23

In [ ]:
# Test Script
comparison_results = []
for i, obs in enumerate(test_cases):
    # Fuzzy system
    firing_res = firings(obs.hr, obs.bp, obs.temp, obs.resp, obs.ox_sats, obs.insp_ox)
    inference_res = inf_eng(firing_res)
    fuzzy_output = de_fuzz(inference_res)
    
    # NEWS2
    news2_score = calculate_news2(obs.hr, obs.bp, obs.temp, obs.resp, obs.ox_sats, obs.insp_ox)
    
    comparison_results.append({
        'Test': i + 1,
        'HR': obs.hr,
        'BP': obs.bp,
        'Temp': obs.temp,
        'Resp': obs.resp,
        'O2': obs.ox_sats,
        'InspO2': obs.insp_ox,
        'NEWS2': news2_score,
        'NEWS2%': round((news2_score / 20) * 100, 1),
        'Fuzzy': round(fuzzy_output, 2)
    })

# Create comparison DataFrame
df_comparison = pd.DataFrame(comparison_results)

print("Fuzzy System vs NEWS2 Score Comparison")
print("=" * 80)
print(df_comparison.to_string(index=False))
print("\n" + "=" * 80)
print(f"Fuzzy Output Range: {df_comparison['Fuzzy'].min():.2f} - {df_comparison['Fuzzy'].max():.2f}")
print(f"NEWS2 Score Range: {df_comparison['NEWS2'].min()} - {df_comparison['NEWS2'].max()}")
print(f"\nCorrelation: {df_comparison['NEWS2'].corr(df_comparison['Fuzzy']):.3f}")

Fuzzy System vs NEWS2 Score Comparison
 Test  HR  BP  Temp  Resp  O2  InspO2  NEWS2  NEWS2%  Fuzzy
    1  75 120  36.8    16  98      21      0     0.0  16.16
    2  80 118  36.5    18  97      21      0     0.0  16.16
    3  70 125  37.0    14  99      21      0     0.0  16.16
    4  52 102  36.2    22  94      30      6    30.0  41.43
    5  95 108  37.5    21  95      21      5    25.0  16.16
    6  48 115  36.0    19  96      21      2    10.0  26.19
    7 105  95  38.2    23  93      28     10    50.0  36.90
    8 115  98  37.8    24  92      35     10    50.0  44.01
    9  42  92  35.8    22  94      40      9    45.0  51.23
   10 100  88  38.5    25  91      40     13    65.0  50.03
   11 125  85  38.8    26  90      50     14    70.0  66.67
   12 135  82  39.2    28  89      60     16    80.0  66.67
   13  38  78  35.2    27  88      50     15    75.0  66.67
   14 145  75  39.5    30  86      70     16    80.0  76.40
   15 150  70  40.0    32  85      80     16    80.0  76.40
 